# Explore & Cluster Neighborhoods in Toronto

## Part I: Data Scraping From Wikipedia

#### Uncomment if following three modules are not already installed.

In [23]:
# !pip install lxml
# !pip install requests
# !pip install beautifulsoup4 

#### Importing necessary python libraries.

In [242]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### Using BeautitulSoup and requests modules to import postal code data around Toronto and convert them into dataframe.

In [143]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
soup = BeautifulSoup(response.content, "lxml")

#### Extract & inspect first table tag and its first 4 rows.

In [144]:
soup.table.find_all('tr')[:4]

[<tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>, <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>]

#### First row is column names and rest are observations. So, extracting first row for column name and rest for rows of dataframe from BeautifulSoup object.

In [145]:
rows = soup.table.find_all('tr')[1:]
columns = soup.table.find('tr').text.split('\n')[1:4]
columns

['Postcode', 'Borough', 'Neighbourhood']

#### Extracting each rows with assigned Borough from the wiki-table & combining Neighborhood with same Postcode.

In [146]:
postcode, borough, neighborhood = [],[],[]
for row in rows:
    _, p,b,n, _ = row.text.split('\n')
    if b.strip().lower() != 'not assigned':
        if p.strip() in postcode:
            i = postcode.index(p.strip())
            neighborhood[i] = neighborhood[i] + ', ' + n.strip()
        else:
            postcode.append(p.strip())
            borough.append(b.strip())
            neighborhood.append(n.strip())

toronto_df = pd.DataFrame(dict(zip(columns,[postcode, borough, neighborhood])))

#### Updating columns name:

In [147]:
toronto_df.columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


#### Let's check size of new dataframe

In [148]:
toronto_df.shape

(103, 3)

# Part II: Adding Lattitude & Longitude

#### Note: Unable to get Lattitude & Longitude using 'Geocoder'. Instead, 'geopy' package can be used but it returns the values slightly different from the values in CSV file at http://cocl.us/Geospatial_data. I am using values in CSV file but the code for downloading geographical coordinates using 'geopy' is provided below.

In [149]:
geo_data = pd.read_csv('https://cocl.us/Geospatial_data')

In [245]:
print(geo_data.shape)
geo_data.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Since geo_data looks sorted by 'Postal Code', let's sort toronto_df by PostalCode.

In [243]:
toronto_df.sort_values('PostalCode', inplace= True)
toronto_df.reset_index(inplace=True, drop=True)

#### Let's check PostalCode of toronto_df is aligned with geo_data.

In [246]:
sum(geo_data['Postal Code'] == toronto_df.PostalCode)

103

#### Lets add Latitude & Longitude columns in toronto_df.

In [153]:
toronto_df['Latitude'] = geo_data.loc[:, 'Latitude']
toronto_df['Longitude'] = geo_data.loc[:, 'Longitude']
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Alternative solution: Code to download Latitude and Longitude using Geopy API

In [154]:
#!pip install geopy  #uncomment if geopy is not installed yet.
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

loc = Nominatim(user_agent="myagent")
geocode = RateLimiter(loc.geocode, min_delay_seconds=1)

In [136]:
#### Uncomment to run the code ####
# temp = toronto_df.Neighborhood.apply(lambda x: geocode(x.split(',')[0] + ', Toronto, Ontario, Canada'))
# toronto_df['Latitude'] = temp.apply(lambda x: None if x is None else x.latitude )
# toronto_df['Longitude'] = temp.apply(lambda x: None if x is None else x.longitude)

# nan_index = temp[temp.values == None].index
# for i in nan_index:
#     next_neighbor = 0
#     neighbor = toronto_df.loc[i, 'Neighborhood']
#     neighbor = neighbor.split(',') if ',' in neighbor else neighbor.split('-')
#     if len(neighbor) > 1: 
#         latlng = geocode(neighbor[0] + ', Toronto, Ontario, Canada')
#         while latlng is None:
#             next_neighbor+=1
#             latlng = geocode(neighbor[next_neighbor] + ', Toronto, Ontario, Canada')            
#     else:
#         latlng = geocode(toronto_df.loc[i, 'Borough'] + ', Toronto, Ontario, Canada')
#     toronto_df.loc[i, 'Latitude'] = latlng.latitude
#     toronto_df.loc[i, 'Longitude'] = latlng.longitude

# Part III: Explore & Cluster Neighborhoods in all Borough of Toronto

In [15]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

### Creating a map of Toronto with neighborhoods in all borough superimposed on top.

In [155]:
latlng = geocode('Toronto, Ontario, Canada')
toronto_lat = latlng.latitude
toronto_lng = latlng.longitude

In [19]:
toronto_lat, toronto_lng

(43.653963, -79.387207)

#### Lets visualize all Toronto neighborhoods on folium map.

In [156]:
toronto_map = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=10)

# adding markers for neighborhoods in each borough
for lat, lng, b, n in zip(toronto_df['Latitude'], toronto_df['Longitude'], 
                          toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(n, b)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map
    

#### Using Foursquare API

In [157]:
CLIENT_ID = 'copy_your_foursquare_CLIENT_ID_here' # your Foursquare ID
CLIENT_SECRET = 'copy_your_foursquare_CLIENT_SECRET_here' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

#### Let's explore the first neighborhood in our dataframe 'df'

In [158]:
lat = toronto_df.loc[0, 'Latitude']
lng = toronto_df.loc[0, 'Longitude']
# Url for accessing foursquare api
url = '''https://api.foursquare.com/v2/venues/explore?&client_id={}
           &client_secret={}&v={}&ll={},{}&radius={}&limit={}'''.format(
                CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
venues_df = pd.io.json.json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues_df = venues_df.loc[:, filtered_columns]

# filter the category for each row to get name
venues_df['venue.categories'] = venues_df['venue.categories'].apply(lambda x: x[0]['name'])

# clean columns
venues_df.columns = [col.split(".")[-1] for col in venues_df.columns]

venues_df

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In [161]:
print('\nThere are {} venues in {} neighborhood.'.format(len(venues_df), toronto_df.loc[0,'Neighborhood']))


There are 2 venues in Rouge, Malvern neighborhood.


#### Lets create a function to process above task of retrieving venues information from foursquare api and converting the result json file into dataframe with desired features.

In [162]:
def getVenues(names, latitude, longitude, radius=500, LIMIT=100):
    venues_list = []
    for name, lat, lng in zip(names, latitude, longitude):
        url = '''https://api.foursquare.com/v2/venues/explore?&client_id={}
                &client_secret={}&v={}&ll={},{}&radius={}&limit={}'''.format(
                CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT) 
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.extend([(name, lat, lng,
                            v['venue']['name'], 
                            v['venue']['location']['lat'], 
                            v['venue']['location']['lng'],  
                            v['venue']['categories'][0]['name']) for v in results
                            ])
    venues_df = pd.DataFrame(venues_list)
    venues_df.columns = ['Neighborhood','Neighborhood Latitude', 'Neighborhood Longitude', 
                          'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return venues_df

### Now, let's explore the neighborhoods in all borough of Toronto.
#### First, let's create new dataframe with the venues around each neighborhoods in the radius of 500.

In [163]:
toronto_venues = getVenues(names=toronto_df['Neighborhood'], 
                           latitude=toronto_df['Latitude'], 
                           longitude=toronto_df['Longitude']
                          )

In [79]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


#### Let's check the size of new dataframe

In [164]:
toronto_venues.shape

(2225, 7)

#### Let's check how many neighborhoods have at least one venue.

In [165]:
print('Following {} Neighborhoods have at least one venue.'.format(
        len(toronto_venues.groupby('Neighborhood').count())))
toronto_venues.groupby('Neighborhood').count()[['Venue Category']]

Following 100 Neighborhoods have at least one venue.


,Venue Category
Neighborhood,
"Adelaide, King, Richmond",100
Agincourt,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",13
"Alderwood, Long Branch",10
"Bathurst Manor, Downsview North, Wilson Heights",20
Bayview Village,4
"Bedford Park, Lawrence Manor East",24
Berczy Park,56


#### Let's find out how many unique categories can be curated from all the returned venues

In [166]:
print('There are {} unique venue categories.'.format(
    len(toronto_venues['Venue Category'].unique())))

There are 267 unique venue categories.


### Analyzing Venues in Each Neighborhood
Encoding categorical feature, 'Venue Category', using get_dummies() function.

In [61]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# Venue Category has 'Neighborhood' venue. To distinguish it from Neighborhood column, 
# 'Neighborhood' venue is renaming to 'Neighborhood Venue'.
toronto_onehot.rename(columns={'Neighborhood': 'Neighborhood Venue'}, inplace=True)
# Adding 'Neighborhood' column from toronto_venue to toronto_onehot
toronto_onehot = pd.concat([toronto_venues.loc[:,'Neighborhood'], toronto_onehot], axis=1)

In [62]:
toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Size of new encoded dataframe vs toronto_venues

In [63]:
toronto_onehot.shape, toronto_venues.shape

((2225, 268), (2225, 7))

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each venue category

In [65]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.

#### Confirming the new size: 100 Neighborhoods have at least one venue and 267 unique venues.

In [66]:
toronto_grouped.shape

(100, 268)

#### Let's print each neighborhood along with the top5 most common venue

In [167]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print(hood)
    print('-' * len(hood))
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Adelaide, King, Richmond
------------------------
              venue  freq
0       Coffee Shop  0.06
1   Thai Restaurant  0.04
2              Café  0.04
3        Steakhouse  0.04
4  Sushi Restaurant  0.03


Agincourt
---------
                       venue  freq
0                     Lounge   0.2
1             Breakfast Spot   0.2
2               Skating Rink   0.2
3             Clothing Store   0.2
4  Latin American Restaurant   0.2


Agincourt North, L'Amoreaux East, Milliken, Steeles East
--------------------------------------------------------
                             venue  freq
0                             Park   0.5
1                       Playground   0.5
2                Accessories Store   0.0
3               Mexican Restaurant   0.0
4  Molecular Gastronomy Restaurant   0.0


Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown
----------------------------------------------------------------------------------------

                   venue  freq
0      Indian Restaurant  0.25
1              Pet Store  0.12
2            Gaming Cafe  0.12
3     Light Rail Station  0.12
4  Vietnamese Restaurant  0.12


Dovercourt Village, Dufferin
----------------------------
      venue  freq
0  Pharmacy  0.13
1    Bakery  0.13
2      Pool  0.07
3       Bar  0.07
4      Bank  0.07


Downsview Central
-----------------
                        venue  freq
0                  Food Truck  0.33
1            Business Service  0.33
2              Baseball Field  0.33
3           Accessories Store  0.00
4  Modern European Restaurant  0.00


Downsview Northwest
-------------------
                  venue  freq
0  Gym / Fitness Center  0.25
1        Discount Store  0.25
2         Grocery Store  0.25
3    Athletics & Sports  0.25
4     Accessories Store  0.00


Downsview West
--------------
                venue  freq
0       Grocery Store  0.50
1                Park  0.25
2                Bank  0.25
3   Accessories Store  0.0

                  venue  freq
0        Massage Studio  0.14
1           Coffee Shop  0.14
2                   Bar  0.14
3  Caribbean Restaurant  0.14
4    Miscellaneous Shop  0.14


Parkdale, Roncesvalles
----------------------
            venue  freq
0       Gift Shop  0.14
1  Breakfast Spot  0.07
2             Bar  0.07
3            Bank  0.07
4         Dog Run  0.07


Parkwoods
---------
                        venue  freq
0           Food & Drink Shop  0.33
1                    Bus Stop  0.33
2                        Park  0.33
3           Accessories Store  0.00
4  Modern European Restaurant  0.00


Queen's Park
------------
                  venue  freq
0           Coffee Shop  0.26
1                   Gym  0.05
2                  Park  0.05
3  Fast Food Restaurant  0.03
4      Sushi Restaurant  0.03


Rosedale
--------
               venue  freq
0               Park  0.50
1         Playground  0.25
2              Trail  0.25
3  Accessories Store  0.00
4      Metro Station  0.00


#### Let's put that into a dataframe
Creating a function to return desire number of top venue names.

In [107]:
def get_top_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Creating a dataframe with top 10 venues for each neighborhood

In [210]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
top_neighborhoods_venues = pd.DataFrame(columns=columns)
top_neighborhoods_venues['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    top_neighborhoods_venues.iloc[ind, 1:] = get_top_venues(toronto_grouped.iloc[ind, :], num_top_venues)

top_neighborhoods_venues.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Thai Restaurant,Steakhouse,Café,Restaurant,Burger Joint,Bakery,Sushi Restaurant,Bar,Cosmetics Shop
1,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Liquor Store,Discount Store,Fried Chicken Joint,Japanese Restaurant,Sandwich Place,Beer Store,Fast Food Restaurant,Pharmacy
4,"Alderwood, Long Branch",Pizza Place,Pub,Pharmacy,Sandwich Place,Dance Studio,Pool,Skating Rink,Coffee Shop,Gym,Comfort Food Restaurant


### Clustering Neighborhoods
Creating KMeans model to cluster neighborhoods into 5 clusters.

In [211]:
knum = 5 # number of clusters
toronto_cluster = toronto_grouped.drop('Neighborhood', axis = 1)
kmean = KMeans(n_clusters=knum, random_state=123).fit(toronto_cluster)
kmean.labels_[:10], len(kmean.labels_)

(array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0]), 100)

#### Creating new dataframe with cluster labels for each neighborhood with top 10 venues

In [212]:
top_neighborhoods_venues.insert(0, 'Cluster Labels', kmean.labels_)
toronto_merged = pd.merge(toronto_df, top_neighborhoods_venues, on='Neighborhood')

#### Checking the size of  new dataframe for proper merge

In [213]:
toronto_merged.shape, toronto_df.shape, top_neighborhoods_venues.shape

((100, 16), (103, 5), (100, 12))

In [214]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Print Shop,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,History Museum,Bar,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dessert Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Electronics Store,Pizza Place,Spa,Breakfast Spot,Medical Center,Intersection,Mexican Restaurant,Rental Car Location,Construction & Landscaping,Convenience Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Lounge,Bakery,Bank,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Fried Chicken Joint,Hakka Restaurant,Drugstore


### Let's visualize the clusters

In [240]:
map_clusters = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=10)

# set color scheme for the clusters
x = np.arange(knum)
ys = [i + x + (i*x)**2 for i in range(knum)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, n, c in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(n) + ' Cluster ' + str(c), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[c-1],
        fill=True,
        fill_color=rainbow[c-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Checking each clusters
#### Cluster 1

In [241]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0,
                   toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",Fast Food Restaurant,Print Shop,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
1,"Highland Creek, Rouge Hill, Port Union",History Museum,Bar,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dessert Shop
2,"Guildwood, Morningside, West Hill",Electronics Store,Pizza Place,Spa,Breakfast Spot,Medical Center,Intersection,Mexican Restaurant,Rental Car Location,Construction & Landscaping,Convenience Store
3,Woburn,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
4,Cedarbrae,Lounge,Bakery,Bank,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Fried Chicken Joint,Hakka Restaurant,Drugstore
6,"East Birchmount Park, Ionview, Kennedy Park",Chinese Restaurant,Coffee Shop,Discount Store,Convenience Store,Department Store,Donut Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
7,"Clairlea, Golden Mile, Oakridge",Bus Line,Bakery,Soccer Field,Ice Cream Shop,Metro Station,Bus Station,Intersection,Park,Gay Bar,Department Store
8,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,Skating Rink,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Yoga Studio
9,"Birch Cliff, Cliffside West",General Entertainment,College Stadium,Café,Skating Rink,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
10,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Gaming Cafe,Thrift / Vintage Store,Chinese Restaurant,Light Rail Station,Pet Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore


#### Cluster 2

In [236]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,
                   toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
90,"Humber Bay, King's Mill Park, Kingsway Park So...",Baseball Field,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market
94,"Emery, Humberlea",Baseball Field,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market


#### Cluster 3

In [237]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,
                   toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Roselawn,Garden,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Deli / Bodega


#### Cluster 4

In [238]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3,
                   toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,York Mills West,Park,Bank,Bar,Convenience Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
24,Parkwoods,Park,Bus Stop,Food & Drink Shop,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Falafel Restaurant
39,East Toronto,Park,Convenience Store,Coffee Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
43,Lawrence Park,Park,Swim School,Bus Line,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Event Space
73,Caledonia-Fairbanks,Park,Pool,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
89,"The Kingsway, Montgomery Road, Old Mill North",Park,River,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
95,Weston,Convenience Store,Park,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


#### Cluster 5

In [239]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4,
                   toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,Jewelry Store,Playground,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
14,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
47,"Moore Park, Summerhill East",Park,Playground,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
49,Rosedale,Park,Trail,Playground,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
